[vla-dev: Way to remotely call F-engine methods atomically #4](https://github.com/realtimeradio/vla-dev/issues/4)
```
"...it is useful to allow remote clients to issue entire CosmicFengine methods in an atomic fashion to ensure that multiple clients to not interfere with one another."
```

In [1]:
class Dummy(object):
		def __init__(self, **kwargs):
			self.dumbness = 'Not at all'
			if 'dumbness' in kwargs:
				self.dumbness = kwargs['dumbness']

		def is_dumb(self, **kwargs):
			if 'dumbness' in kwargs:
				self.dumbness = kwargs['dumbness']
			return self.dumbness

		def add(self, a: int, b: int):
			return a + b


In [2]:
dunce = Dummy()
# It's possible to get methods by name...
method = getattr(dunce, 'is_dumb')
method(dumbness = 'See, not so...')

# ...and they operate on their instance
dunce.is_dumb()

'See, not so...'

In [3]:
# Using the standard inspect package, it's possible to get all methods of an instance
import inspect
[method for method in inspect.getmembers(dunce, lambda a: inspect.isroutine(a))]
# and filter out what should be private

[('__dir__', <function Dummy.__dir__()>),
 ('__format__', <function Dummy.__format__(format_spec, /)>),
 ('__init__',
  <bound method Dummy.__init__ of <__main__.Dummy object at 0x000001F8AA1C39A0>>),
 ('__init_subclass__', <function Dummy.__init_subclass__>),
 ('__new__', <function object.__new__(*args, **kwargs)>),
 ('__reduce__', <function Dummy.__reduce__()>),
 ('__reduce_ex__', <function Dummy.__reduce_ex__(protocol, /)>),
 ('__sizeof__', <function Dummy.__sizeof__()>),
 ('__subclasshook__', <function Dummy.__subclasshook__>),
 ('add',
  <bound method Dummy.add of <__main__.Dummy object at 0x000001F8AA1C39A0>>),
 ('is_dumb',
  <bound method Dummy.is_dumb of <__main__.Dummy object at 0x000001F8AA1C39A0>>)]

In [4]:
# Furthermore, it's possible to get the signature of the methods...
method = getattr(dunce, 'add')
{key: parameter for (key, parameter) in inspect.signature(method).parameters.items()}

{'a': <Parameter "a: int">, 'b': <Parameter "b: int">}

So the server Python instance would be able to provide a single `call-this-method-with-these-arguments` endpoint.

The client Python instance would be able to query available methods and their signatures...

In [5]:
# Python can write dynamic class definitions...
classname = 'Dummy'
loc = [
	f"class {classname}Mock(object):",
	f"\tdef __init__(self):",
	f"\t\tpass",
	f"",
]
print('\n'.join(loc))

class DummyMock(object):
	def __init__(self):
		pass



In [6]:
# ... definitions can be executed and added to the environment
local_dict = {}
exec('\n'.join(loc), None, local_dict)
globals()[f"{classname}Mock"] = local_dict[f"{classname}Mock"]

mock = DummyMock()

In [7]:
# Dynamic functions can also be defined
loc = [
f"def add(self, a, b):",
"\tprint(f'DummyMock<{hex(id(self))}>.add(a: {a}, b: {b});')"
]

local_dict = {}
exec('\n'.join(loc), None, local_dict)
mock_add_func = local_dict["add"]
mock_add_func(mock, 11, 31)

DummyMock<0x1f8a9e771c0>.add(a: 11, b: 31);


In [8]:
# And a function can be set as an instance's method
import types
setattr(mock, 'add', types.MethodType(mock_add_func, mock))

mock.add(11, 31)

DummyMock<0x1f8a9e771c0>.add(a: 11, b: 31);


That's fundamentally how `remoteobjects` provides native-access to remote python-objects under a client-server model.

- A REST server exposes registration of and (method/attribute) access to instances of classes

- Client-local classes are dynamically defined with signatures matching their remote targets.
- Client-local classes have methods/properties that make REST requests to their remote counterparts (methods/attributes)